In [21]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_nmansou4" #please modify so you store data in your collection
my_char = 'u'

# beginning page index
begin = "1"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]


gitlab_url = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

gleft = 20

source_url = "https://sourceforge.net/directory/?q=" + my_char + "&sort=name&page="
rest_url = "https://sourceforge.net/rest/p/"

header = {'per_page': 99}

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

def project_exists(url):
    r = requests.get(url)
    if r.status_code == 200:
        return True
    return False

def get_source(url, coll, rest):
    page = 1
    project_count = 0
    while True:
        resp = requests.get(url + str(page))
        text = resp.text
        soup = BeautifulSoup(text, 'html.parser')
        if re.search('No results found.', soup.get_text()):
            return

        for link in soup.find_all(class_="project-icon", href=True):
            name = re.findall('/projects/([A-Za-z0-9\-]*)', link.get('href'))
            name = name[0] if name else None
            if name is not None and name.lower().startswith(my_char):
                resp = requests.get(rest + name)
                if resp.status_code == 200:
                    info = json.loads(resp.text)
                    info['forge'] = 'sourceforge'
                    coll.insert_one(info)
                    project_count += 1
                    if project_count >= 50:
                        return
        page += 1
    return

# send queries and extract urls 
def get_gitlab(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    project_count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if el['name'].lower().startswith(my_char):
                    if project_exists(el['http_url_to_repo']):
                        project_count += 1
                        el['forge'] = 'gitlab'
                        coll.insert_one(el)
                        if project_count >= 50:
                            return
            
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(my_char):
                                if project_exists(el['http_url_to_repo']):
                                    project_count += 1
                                    el['forge'] = 'gitlab'
                                    coll.insert_one(el)
                                    if project_count >= 50:
                                        return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')
            
        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
#start retrieving        
get_gitlab(gitlab_url,coll)
get_source(source_url, coll, rest_url)
#print collected data
for doc in coll.find({}):
    print(doc)

{'forks_count': 0, 'path_with_namespace': 'Synvie/umx-video-list', 'star_count': 0, 'id': 8834180, 'path': 'umx-video-list', 'http_url_to_repo': 'https://gitlab.com/Synvie/umx-video-list.git', 'namespace': {'kind': 'user', 'path': 'Synvie', 'parent_id': None, 'full_path': 'Synvie', 'id': 3660961, 'name': 'Synvie'}, 'description': 'This is an uMatrix list to allow embedded video-sharing website.', 'default_branch': 'master', 'ssh_url_to_repo': 'git@gitlab.com:Synvie/umx-video-list.git', 'forge': 'gitlab', 'tag_list': [], 'name_with_namespace': 'Synvie / uMatrix Video List', '_id': ObjectId('5bc0cc5881c12569d1a2d74c'), 'last_activity_at': '2018-10-12T16:16:01.102Z', 'created_at': '2018-10-12T16:16:01.102Z', 'avatar_url': None, 'readme_url': 'https://gitlab.com/Synvie/umx-video-list/blob/master/README.md', 'name': 'uMatrix Video List', 'web_url': 'https://gitlab.com/Synvie/umx-video-list'}
{'forks_count': 0, 'path_with_namespace': 'philbooth/unicode-bom', 'star_count': 0, 'id': 8831918, '

In [20]:
##deleting the collection for avoiding duplicate keys when runnung the script again
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_nmansou4" #please modify so you store data in your collection
my_char = 'u'

# beginning page index
begin = "1"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]

result= my_collection.delete_many({})


In [45]:
#print gitlab projects
git = 1
source = 1
for info in coll.find({}):
    if("web_url" in info):
        print("gitLab ", git, info["web_url"])
        git+=1
    else:
        print("sourceforge ", source, info["url"])
        source+=1


gitLab  1 https://gitlab.com/Synvie/umx-video-list
gitLab  2 https://gitlab.com/philbooth/unicode-bom
gitLab  3 https://gitlab.com/juliandavid677/unomi
gitLab  4 https://gitlab.com/it25/unix_setup
gitLab  5 https://gitlab.com/arifjaunpur/userlist
gitLab  6 https://gitlab.com/mash_33/userdatenbank
gitLab  7 https://gitlab.com/florianweg/usaltmann.theme
gitLab  8 https://gitlab.com/undercarriage/undercarriage.gitlab.io
gitLab  9 https://gitlab.com/svv1313/url-shortenizer
gitLab  10 https://gitlab.com/wiwo-jurian/uhh
gitLab  11 https://gitlab.com/kat.morgan/ubiquiti-lxd-ctl-server
gitLab  12 https://gitlab.com/hellekin/uuid_parameter
gitLab  13 https://gitlab.com/ragulkanth/usbnotify
gitLab  14 https://gitlab.com/avan1235/U8g2_Arduino
gitLab  15 https://gitlab.com/bhdouglass/uappexplorer
gitLab  16 https://gitlab.com/EnricRoda/uas-project
gitLab  17 https://gitlab.com/Lulzx/ubiquitous-telegram
gitLab  18 https://gitlab.com/Brecka/unitycitest
gitLab  19 https://gitlab.com/colab-br/ususpres